# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [285]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

%matplotlib inline

In [286]:
# load messages dataset
messages = pd.read_csv('./data/disaster_messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [287]:
# Print info about dataframe messages
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
id          26248 non-null int64
message     26248 non-null object
original    10184 non-null object
genre       26248 non-null object
dtypes: int64(1), object(3)
memory usage: 820.3+ KB


In [288]:
# Print shape of dataframe messages
messages.shape

(26248, 4)

In [289]:
# Print unique values of genre
messages['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

In [290]:
# Print value counts of each genre
messages['genre'].value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

In [291]:
messages['genre'].value_counts().sum()

26248

In [292]:
# Check missing values in dataframe messages
messages.isnull().any()

id          False
message     False
original     True
genre       False
dtype: bool

In [293]:
# Print first 10 observations where original has a null value
messages[messages['original'].isnull()].head(10)

,id,message,original,genre
7433,8365,NOTES: It mark as not enough information,NaN,direct
9902,11186,My thoughts and prayers go out to all the live...,NaN,social
9903,11188,I m sorry for the poor people in Haiti tonight...,NaN,social
9904,11189,RT selenagomez UNICEF has just announced an em...,NaN,social
9905,11192,lilithia yes 5.2 magnitude earthquake hit mani...,NaN,social
9906,11193,RT TheNewsBlotter RT caribnews On Call Interna...,NaN,social
9907,11195,Most Eureka homeowners won&#39 t collect on qu...,NaN,social
9908,11196,Haiti hit by largest earthquake in over 200 ye...,NaN,social
9909,11197,RT pinkelephantpun Earthquake Relief Donate ht...,NaN,social
9910,11198,Praying that today s earthquake in #Haiti mini...,NaN,social


In [294]:
# Print count of observations where original has null value
messages[messages['original'].isnull()].shape[0]

16064

From above, we observe that only column 'original' in dataframe messages have null values and there are 16064 observations with null values in column 'original'.

In [295]:
# load categories dataset
categories = pd.read_csv('./data/disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [296]:
# Print info about dataframe categories
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 2 columns):
id            26248 non-null int64
categories    26248 non-null object
dtypes: int64(1), object(1)
memory usage: 410.2+ KB


In [297]:
# Print shape of dataframe categories
categories.shape

(26248, 2)

In [298]:
# Print unique values in column categories of dataframe categories
categories.loc[0,'categories'].split(';')

['related-1',
 'request-0',
 'offer-0',
 'aid_related-0',
 'medical_help-0',
 'medical_products-0',
 'search_and_rescue-0',
 'security-0',
 'military-0',
 'child_alone-0',
 'water-0',
 'food-0',
 'shelter-0',
 'clothing-0',
 'money-0',
 'missing_people-0',
 'refugees-0',
 'death-0',
 'other_aid-0',
 'infrastructure_related-0',
 'transport-0',
 'buildings-0',
 'electricity-0',
 'tools-0',
 'hospitals-0',
 'shops-0',
 'aid_centers-0',
 'other_infrastructure-0',
 'weather_related-0',
 'floods-0',
 'storm-0',
 'fire-0',
 'earthquake-0',
 'cold-0',
 'other_weather-0',
 'direct_report-0']

In [299]:
# Count of unique categories
len(categories.loc[0,'categories'].split(';'))

36

In [300]:
# Print categories
category_list = []
for category in categories.loc[0,'categories'].split(';'):
    category_list.append(category.split('-')[0])
category_list

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [301]:
# defining function cat_count to count the number of categories in each observation of dataframe categories
def cat_count(cat_str):
    '''count the number of categories in each observation of dataframe

    INPUT - 
            cat_str - str, value of column categories in each observation
    OUTPUT -
            number - int, number of categories contained in each observation
    
    '''
    
    cat_list = []
    for cat in cat_str.split(';'):
        cat_list.append(cat.split('-')[0])
    return len(cat_list)

In [302]:
# Find category count of each observation
category_count = categories['categories'].apply(lambda x: cat_count(x))
print(type(category_count),'\n')
print(category_count.head())

<class 'pandas.core.series.Series'> 

0    36
1    36
2    36
3    36
4    36
Name: categories, dtype: int64


In [303]:
# Print unique values in Series category_count
category_count.unique()

array([36], dtype=int64)

In [304]:
# Print value counts of each unique values in Series category_count
category_count.value_counts()

36    26248
Name: categories, dtype: int64

From above, we observe that each observation in dataframe categories have 36 categories mentioned in column 'categories'.

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [305]:
# merge datasets
df = messages.merge(categories, how='inner', on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [306]:
# print shape of merged dataframe
df.shape

(26386, 5)

In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26386 entries, 0 to 26385
Data columns (total 5 columns):
id            26386 non-null int64
message       26386 non-null object
original      10246 non-null object
genre         26386 non-null object
categories    26386 non-null object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [308]:
# create a dataframe of the 36 individual category columns
categories = categories['categories'].str.split(';', expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [309]:
# select the first row of the categories dataframe
row = categories.iloc[0,:]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda x: x[:-2])
category_colnames = category_colnames.tolist()
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [310]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [311]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1]
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [312]:
# Confirm values in each column of the categories dataframe are, in fact, only 0's and 1's
# And if not, print column name along with value counts of each unique values

for column in categories:
    if set(categories[column].unique()) != set([0,1]):
#        print(column, set(categories[column].unique()))
        print('Column {} has unique values as {}'.format(column, set(categories[column].unique())))
        print(categories[column].value_counts(),'\n')

Column related has unique values as {0, 1, 2}
1    19930
0     6125
2      193
Name: related, dtype: int64 

Column child_alone has unique values as {0}
0    26248
Name: child_alone, dtype: int64 



In [313]:
categories[categories['related'] == 2].shape

(193, 36)

In [314]:
categories[categories['related'] == 2].head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [315]:
# Check sum of values in all columns after filtering dataframe categories
# having observations with column "related" with value as 2
categories[categories['related'] == 2].sum()

related                   386
request                     0
offer                       0
aid_related                 0
medical_help                0
medical_products            0
search_and_rescue           0
security                    0
military                    0
child_alone                 0
water                       0
food                        0
shelter                     0
clothing                    0
money                       0
missing_people              0
refugees                    0
death                       0
other_aid                   0
infrastructure_related      0
transport                   0
buildings                   0
electricity                 0
tools                       0
hospitals                   0
shops                       0
aid_centers                 0
other_infrastructure        0
weather_related             0
floods                      0
storm                       0
fire                        0
earthquake                  0
cold      

From above, we observe that there are 193 observations where the disaster message has a category "related" with value as 2. Also, we can see that these 193 observations have values only in column "related" and rest all columns have values as 0. We would analyze these observations later on and determine what to do with them in the next section.

Also, we observe that category "child_alone" has only one value i.e. 0 in all the observations which means all the disaster messages had category "child_alone" value as 0 which indicates that none of the disaster message corresponds to child being alone. Since column 'child_alone' has only value, it will not be good data to include in our machine learning model later on to evaluate if a child is alone or not. By using this data, our machine learning model will only learn that child is not alone always which would not be true every time for any new disaster message. Therefore, dropping column "child_alone".

In [316]:
# Drop child_alone from categories dataframe.
categories.drop('child_alone', axis = 1, inplace = True)
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [317]:
# drop the original categories column from `df`

df.drop('categories', axis=1, inplace=True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [318]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1, join='inner')
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26248 entries, 0 to 26247
Data columns (total 39 columns):
id                        26248 non-null int64
message                   26248 non-null object
original                  10246 non-null object
genre                     26248 non-null object
related                   26248 non-null int64
request                   26248 non-null int64
offer                     26248 non-null int64
aid_related               26248 non-null int64
medical_help              26248 non-null int64
medical_products          26248 non-null int64
search_and_rescue         26248 non-null int64
security                  26248 non-null int64
military                  26248 non-null int64
water                     26248 non-null int64
food                      26248 non-null int64
shelter                   26248 non-null int64
clothing                  26248 non-null int64
money                     26248 non-null int64
missing_people            26248 non-null i

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [320]:
# check number of duplicates
#df.duplicated(subset=None, keep='first').sum()
print('Number of duplicated rows: {} out of {} observations'.format(df.duplicated(subset=None, keep='first').sum(),\
                                                               df.shape[0]))


Number of duplicated rows: 41 out of 26248 observations


In [321]:
# drop duplicates
df.drop_duplicates(subset=None, keep='first', inplace=True)


In [322]:
# check number of duplicates
print('Number of duplicated rows after droping duplicates: {} out of {} observations'.\
      format(df.duplicated(subset=None, keep='first').sum(), df.shape[0]))

Number of duplicated rows after droping duplicates: 0 out of 26207 observations


### 7. Analyze  disaster messages with category column "related" having value as 2.

In [323]:
# Get disaster messages with category "related" having value as 2
df[df['related'] == 2]['message']

117      Dans la zone de Saint Etienne la route de Jacm...
219      Hygienic . .. , mobile toilets, batteries, gen...
305      I am a surviver of the earthkaque see how you ...
460      I am located in Route Freres- Impasse Fortin. ...
576      Please people in Carrefour, Magloire Ambroide ...
655      People in grand goave, especially n Foch. we a...
656      A lot of victims arrive at the hospital Erez M...
883      oh please we are in the need of food. water. w...
897      And where will I make contact with you? Thank ...
925          I do not have a card for me to call my family
931      we sleep with the baby. Thanks in advance for ...
933      We are living in deplorable conditions. We are...
1228     I'm in/at Kot Plaj ( beach ) 22#120, my house ...
1250     Carefour Feiulles zone Savane Pistache. we hav...
1311     I would like to know if it's true that there i...
1403      I would like some information on the earthquake.
1498     Please, do something for us, we are among near.

Many of the disaster messages with category "related" having value as 2 appear to be in languages other than English. Based on this, we infer that a related value of 2 corresponds to uncertainty as to whether or not the message is related. As a result, these observations are not really useful when it comes to building an ML model, so we will drop these records from the dataset.

In [324]:
# Drop observations with column "related" having value as 2
df.drop(df.index[df['related'] == 2].tolist(), axis=0, inplace=True)

In [325]:
df.shape

(26015, 39)

### 8. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df.to_sql('InsertTableName', engine, index=False)

### 9. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.